In [1]:
from rberga06.phylab import Datum as d

V0 = (d(.076, .001) + d(.088, .001) + d(.086, .001) + d(.091, .001))/4
# V0 = d(.0032, 7e-4)
print(V0, "V")

Datum(best=0.08524999999999999, delta=0.001) V


In [2]:
# Regressione lineare
#    V = AT + B
# => T = (V - B)/A
A1 = d(0.0033371828, 4.9302375173E-6)
B1 = d(-0.8735026125, 0.0014783199)
T1 = (V0 - B1)/A1
print(T1 - d(273.15, .01), "°C")

Datum(best=14.144005141102923, delta=1.1770764881810023) °C


In [3]:
# Regressione quadratica
#    V = AT² + BT + C
# => T = (-B + √(B² - 4AC))/2A
A2 = d(3.1345109092E-6, 4.0664944486E-8)
B2_2 = d(0.0018186742, 2.0307628607E-5)/2
C2 = d(-0.7260940987, 0.0024171484)
Delta_4 = B2_2**2 - A2*C2
T2 = (Delta_4**.5 - B2_2)/A2
print(T2, "K")
print(T2 - d(273.15, .01), "°C")

Datum(best=271.86123148674096, delta=11.79849731503083) K
Datum(best=-1.288768513259015, delta=11.808497315030829) °C


In [34]:
# Regressione cubica
#   f(T)  = AT³ + BT² + CT + D - V
#   f'(T) = 3AT² + 2BT + C
#  Metodo di Newton (delle tangenti)
#   0 - f(Tₙ) = f'(Tₙ)(Tₙ₊₁ - Tₙ)
#   Tₙ₊₁ = Tₙ - f(Tₙ)/f'(Tₙ)
A3 = d(-9.4026125488E-9, 8.0213829342E-10)
B3 = d(1.0335859351E-5, 6.1569249727E-7)
C3 = d(2.3272301055E-4, 13681329502E-4)
D3 = d(-0.6407138592, 0.0076743967)
_3A = 3 * A3
_2B = 2 * B3
T3 = d(300, 0)
for i in range(10):
    print(i, T3, "K")
    #print(i, T3 - d(273.15, .01), "°C")
    T3 = T3 - (A3 * T3 ** 3 + B3 * T3 ** 2 + C3 * T3 + D3 - V0)/(_3A * T3**2 + _2B * T3 + C3)
print(T3, "K")
print(T3 - d(273.15, .01), "°C")

0 Datum(best=300, delta=0) K
1 Datum(best=294.69779564361926, delta=33.61665238440816) K
2 Datum(best=294.6838229759427, delta=141.54251568288186) K
3 Datum(best=294.68382287387965, delta=493.7493669187135) K
4 Datum(best=294.68382287387965, delta=1643.1874597453955) K
5 Datum(best=294.68382287387965, delta=5394.414685804405) K
6 Datum(best=294.68382287387965, delta=17636.662258199918) K
7 Datum(best=294.68382287387965, delta=57589.62074141024) K
8 Datum(best=294.68382287387965, delta=187977.35241581406) K
9 Datum(best=294.68382287387965, delta=613501.7995813661) K
Datum(best=294.68382287387965, delta=2002214.16392948) K
Datum(best=21.53382287387967, delta=2002214.17392948) °C
